## Example of using the Abra

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
import statsmodels.api as sm
%matplotlib inline

In [44]:
from transformations import *
from selection import *

### Applying transforms

This example uses the naics5811 [dataset](http://www.nber.org/nberces/). A description of the columns is [here](http://www.nber.org/nberces/nberces5811/nberces_5811_summary_stats.pdf).

In [4]:
data = pd.read_csv('~/Downloads/naics5811.csv')
data = data.drop(['naics'], axis=1)
train, test = train_test_split(data,test_size=0.4)

get_standard_transforms_for_dataset creates a list of dicts describing standard transformations (e.g. imputations, caps, floors, onehot encoding, ihs transforms) and their parameters for the dataset columns. This list can be stored as json and used to implement the model.

In [5]:
standard_transforms = get_standard_transforms_for_dataset(train)

ihs_transforms = get_ihs_transforms_for_dataset(
    apply_transforms(train, standard_transforms), 
    drop_input=True
)

angular_transforms = [
    get_transform('pimat_ang', 'pimat', 'angular', drop_input=True),
    get_transform('piinv_ang', 'piinv', 'angular', drop_input=True),
    get_transform('pien_ang', 'pien', 'angular', drop_input=True)
]

transforms = standard_transforms+ihs_transforms+angular_transforms

In [6]:
transforms[50]

{'output': 'prode',
 'input': 'prode',
 'func': 'cap',
 'params': {'cutoff': 846.3401709195086},
 'drop_input': False}

In [7]:
transforms[120]

{'output': 'plant_ihs',
 'input': 'plant',
 'func': 'ihs',
 'params': {'ihs_factor': 10.69928324087381},
 'drop_input': True}

The list of dicts is then used by apply_transforms to apply each transformation to both the test and train set.

In [8]:
train = apply_transforms(train, transforms)
test = apply_transforms(test, transforms)

After that get_correlated_columns and get_unary_columns can be used to remove redundant and or un-informative columns.

In [9]:
correlated_column_sets = get_correlated_columns(train)

for i in range(len(correlated_column_sets)):
    test['col_set_{}'.format(i)] = test[correlated_column_sets[i][0]]
    train['col_set_{}'.format(i)] = train[correlated_column_sets[i][0]]
    test = test.drop(correlated_column_sets[i], axis=1)
    train = train.drop(correlated_column_sets[i], axis=1)

In [10]:
unary_columns = get_unary_columns(train)
train = train.drop(unary_columns, axis=1)
test = test.drop(unary_columns, axis=1)

train['const'] = 1
test['const'] = 1

### Variable Reduction

Next variable reduction procedures can be used to select the model. In this example the target is the total capex (invest).

In [11]:
target = 'invest_ihs'
X = train.copy().drop(target, axis=1)
y = train[target]

In [12]:
all_cols = list(X)
model = sm.OLS(train[target], train[all_cols]).fit()

In [13]:
print('Train R^2: {}'.format(r2_score(train[target],model.predict(train[all_cols]))))
print('Test R^2: {}'.format(r2_score(test[target],model.predict(test[all_cols]))))

Train R^2: 0.9323008236961887
Test R^2: 0.9295809035673137


One selection procedure that can be used is bayesian model selection (bms in R). Note, only linear regression is supported, but for selection linear regression can be used on binary targets.

In [24]:
bms_vars, pip, _, _ = fast_bms(X, y, iterations=10000, prior=get_binomial(25, 0.1))
pip.head(10)

,Variables,PIP
0,year,1.0
16,emp_ihs,1.0
34,pien_ang,1.0
33,piinv_ang,1.0
31,tfp4_ihs,1.0
29,piship_ihs,1.0
28,plant_ihs,1.0
27,equip_ihs,1.0
26,cap_ihs,1.0
25,energy_ihs,1.0


In [15]:
model = sm.OLS(train['invest_ihs'], train[bms_vars]).fit()
print('Train R^2: {}'.format(r2_score(train['invest_ihs'], model.predict(train[bms_vars]))))
print('Test R^2: {}'.format(r2_score(test['invest_ihs'], model.predict(test[bms_vars]))))

Train R^2: 0.9322376842430504
Test R^2: 0.9295354494173846


Another is selection procedure is backward_selection. Similar to fast_bms, backward_selection only supports linear regression, but produces reasonable results for binary targets. Use fast_backward_selection to use logistic regression (which uses the wald test to determine which columns to drop at each iteration).

In [26]:
results = backward_selection(X, y)
bs_vars = results[0]
results[1].head(10)

,BIC,AIC,Adj. R-squared,Dropped Variables,RSS
0,15532.948191,15242.733042,0.932133,,2413.990155
1,15523.309286,15240.731378,0.932137,[12],2413.989893
2,15513.673796,15238.733129,0.932141,[3],2413.990169
3,15504.047918,15236.744491,0.932146,[14],2413.991959
4,15494.491017,15234.824831,0.932150,[15],2414.004614
5,15485.029730,15233.000785,0.932154,[10],2414.032331
6,15475.757325,15231.365621,0.932156,[32],2414.089801
7,15466.481988,15229.727524,0.932159,[11],2414.146811
8,15457.566714,15228.449492,0.932160,[1],2414.260545
9,15449.630346,15228.150364,0.932157,[9],2414.528511


In [17]:
model = sm.OLS(train['invest_ihs'], train[bs_vars]).fit()
print('Train R^2: {}'.format(r2_score(train['invest_ihs'],model.predict(train[bs_vars]))))
print('Test R^2: {}'.format(r2_score(test['invest_ihs'],model.predict(test[bs_vars]))))

Train R^2: 0.9321714258860556
Test R^2: 0.9295290483194992


### Group Selection

The selection functions also support group selection. A use case of this, is including square terms and requiring the selection functions to include both the original variable and square term in the model if the square term is selected.

In this example, square terms are created using get_square_transforms_for_dataset.

In [18]:
X_r = X[bs_vars]
square_transforms = get_square_transforms_for_dataset(X_r)

In [19]:
X_r = apply_transforms(X_r, square_transforms)
train = apply_transforms(train, square_transforms)
test = apply_transforms(test, square_transforms)

To use group selection, a group matrix needs to passed. The ij-th entry of the group matrix is 1 if variable
j must be included with variable i and 0 otherwise. get_group_matrix can be used to create the group matrix, using the dataset and a group_dict where the values are lists of variables that must be included in the model if the corresponding key is in the model.

In [20]:
group_dict = get_group_dict(square_transforms)
group_matrix = get_group_matrix(X_r, group_dict)

group_dict

{'year_sqr': ['year'],
 'dtfp4_sqr': ['dtfp4'],
 'emp_ihs_sqr': ['emp_ihs'],
 'pay_ihs_sqr': ['pay_ihs'],
 'prode_ihs_sqr': ['prode_ihs'],
 'prodh_ihs_sqr': ['prodh_ihs'],
 'prodw_ihs_sqr': ['prodw_ihs'],
 'vship_ihs_sqr': ['vship_ihs'],
 'matcost_ihs_sqr': ['matcost_ihs'],
 'vadd_ihs_sqr': ['vadd_ihs'],
 'energy_ihs_sqr': ['energy_ihs'],
 'cap_ihs_sqr': ['cap_ihs'],
 'equip_ihs_sqr': ['equip_ihs'],
 'plant_ihs_sqr': ['plant_ihs'],
 'piship_ihs_sqr': ['piship_ihs'],
 'tfp5_ihs_sqr': ['tfp5_ihs'],
 'tfp4_ihs_sqr': ['tfp4_ihs'],
 'piinv_ang_sqr': ['piinv_ang'],
 'pien_ang_sqr': ['pien_ang']}

In [21]:
result = backward_selection(X_r, y, group_matrix=group_matrix)
bs_vars_w_sqr = result[0]

In [22]:
model = sm.OLS(train['invest_ihs'], train[bs_vars_w_sqr]).fit()
print('Train R^2: {}'.format(r2_score(train['invest_ihs'],model.predict(train[bs_vars_w_sqr]))))
print('Test R^2: {}'.format(r2_score(test['invest_ihs'],model.predict(test[bs_vars_w_sqr]))))

Train R^2: 0.9340769916055198
Test R^2: 0.931257287398192


Another use of group selection is using b-splines.

In [27]:
X_r = X[bs_vars]

In [28]:
bspline_transforms = get_bspline_transforms_for_dataset(X_r, df=3, degree=2)
X_r = apply_transforms(X_r, bspline_transforms)
train = apply_transforms(train, bspline_transforms)
test = apply_transforms(test, bspline_transforms)

group_matrix = get_group_matrix(X_r, get_group_dict(bspline_transforms))

In [29]:
t = backward_selection(X_r, y, group_matrix=group_matrix)
bs_vars_w_spline = t[0]

In [30]:
model = sm.OLS(train['invest_ihs'], train[bs_vars_w_spline]).fit()
print('Train R^2: {}'.format(r2_score(train['invest_ihs'],model.predict(train[bs_vars_w_spline]))))
print('Test R^2: {}'.format(r2_score(test['invest_ihs'],model.predict(test[bs_vars_w_spline]))))

Train R^2: 0.9349460070644586
Test R^2: 0.9320557617838351


### Using fast_backward_selection

In this example, an artificial dataset is used to demonstrate the use of fast_backward_selection, which supports regression using any statsmodel object.

In [65]:
X, y, coeff = sklearn.datasets.make_regression(100000, 100, 20,  noise=500, coef=True)
y = 1*(1/(1+np.exp(-(y-3*y.std())/y.std()))>np.random.rand(100000))

grp = np.eye(100)
grp[:, 99] = 1
grp[:, 98] = 1

In [55]:
fbw_result = fast_backward_selection(
    pd.DataFrame(X), 
    pd.Series(y),
    sm.GLM,model_kwargs={'family':sm.families.Binomial()},
    criteria='bic',
    refit_freq=2, 
    group_matrix=grp
)

In [59]:
fbw_result[1].head(10)

,Dropped Variables,BIC,AIC,Max p-value
0,[],-77468.790776,13913.578906,0.163514
1,"[3, 61]",-77487.208838,13909.581525,0.163610
2,"[36, 29]",-77505.628207,13905.582837,0.163342
3,"[93, 94]",-77524.037232,13901.594493,0.163606
4,"[58, 84]",-77542.452571,13897.599834,0.163101
5,"[72, 53]",-77560.837313,13893.635773,0.163690
6,"[56, 89]",-77579.224955,13889.668812,0.163570
7,"[59, 13]",-77597.599887,13885.714560,0.164072
8,"[0, 75]",-77615.976477,13881.758652,0.163748
9,"[88, 50]",-77634.340598,13877.815211,0.163400
